In [60]:
import os
import csv
import json
from bs4 import BeautifulSoup
import shutil
import requests
import pandas as pd
import re

In [61]:
recipes=[]

In [62]:
def get_cooking_time(soup):
    duration=""
    facts=soup.find_all('div',{'class':'facts'})
    for fact in facts:
        time=fact.find('div',{'class':'facts__item'})
        time1=time.find('dd',{'class':'facts__value'})
        duration=duration+time1.get_text()
    return duration

In [63]:

def get_ingredients(soup):
    ingredient=soup.find_all('ul',{'class':'ingredient-list'})
    ingredients=""
    for i in ingredient:
        ingri=i.find_all('li')
        for li in ingri:
            if(li.find('h4')):
                continue
            text=""
            for span in li.find_all('span',{'class':'ingredient-text'}):
               text=text+span.get_text(separator=' ', strip=True) + ' '
            for quantity in li.find_all('span',{'class':'ingredient-quantity'}):
                text=quantity.get_text()+' '+text
            ingredients=ingredients+text.strip()+"\n"
    return ingredients

In [64]:
def get_directions(soup):
    curr_recipe=soup.find('div',{'class':'current-recipe'})
    directions=""
    ul=curr_recipe.find('ul',{'class':'direction-list'})
    li=ul.find_all('li',{'class':'direction'})
    for direction in li:
        list_item_text=direction.text.encode('ascii','ignore')
        directions+=list_item_text.decode() + "\n"
    return directions
        
        

In [65]:
def get_servings(soup):
    servings=""
    facts=soup.find_all('div',{'class':'facts'})
    for fact in facts:
        items=fact.find_all('div',{'class':'facts__item'})
        for item in items:
            serving=item.find('dd',{'class':'facts__value'})
            if(serving.find('span',{'class':'value'})):
                servings=servings+serving.find('span',{'class':'value'}).get_text()
            else:
                continue
    return servings

In [66]:
def get_calories(soup):
    script_tag = soup.find('script', type='application/ld+json')
    if script_tag is not None:
        calories_json = script_tag.string
        if calories_json is not None:
            calories = json.loads(calories_json).get("nutrition", {}).get("calories")
            return calories
    return None

In [67]:
def text_scrape(recipe_name,url):
    data=requests.get(url)
    soup=BeautifulSoup(data.content,'html.parser')
    dicto={'name':recipe_name,'cooking_time':get_cooking_time(soup),'calories':get_calories(soup),'ingredients':get_ingredients(soup),'directions':get_directions(soup),'serving':get_servings(soup)}
    return dicto

In [ ]:
def main():
    df=pd.read_csv(r"C:\Users\aditi\OneDrive\Desktop\DeepChef Personal\links_copy_main.csv")
    for index, row in df.iloc[667:778].iterrows():
        recipes.append(text_scrape(row['name'],row['link']))
    with open('recipes.json','r+') as f:
        json.dump(recipes,f)

if __name__=="__main__":
    main()

In [69]:
with open("recipes.json", 'r',encoding="utf8") as f:
    data = f.read()

# Regular expression to match "\u2044" preceded and followed by digits
pattern = r"(\d)\\u2044(\d)" #1 1\u20442
# Replace matches with "numerator/denominator" format
converted_data = re.sub(pattern, r"\1/\2", data)

# Write the converted data back to the file
with open("recipes.json", 'w') as f:
    f.write(converted_data)